# GraphVAE for networks generation

### Variational Autoencoders

First of all let's have a brief remind on **VAE** Variational autoencoders, firstly introduced by [*D. P. Kingma and M. Welling. 'Auto-encoding variational bayes', 2014*](https://arxiv.org/pdf/1312.6114.pdf)

VAE is a neural network architecture belonging to the family of variational Bayesian methods.

From a probabilistic point of view we want to maximize the likelyhood of our data **x** given a proper set of parameters **$\theta$**, like in a normal MLE problem: $p_{\theta}(x) = p(x|\theta)$. By neglecting from the third moment upwards, we could approximate the distribution to a normal distribution $\mathcal{N}(x|\mu,\sigma)$. Simple distributions like the normal ones are usually easy to maximize, however if we assume a prior over a latent space $z$ the posterior usually becomes intractable.

By marginalizing over $z$ we obtain:

$$p_{\theta}(x) = \int_{\mathcal{Z}}{p_{\theta}(x,z)dz} = \int_{\mathcal{Z}}{p_{\theta}(x|z)p_{\theta}(z)dz}$$

So we may define the set of relationships between the input data and the latent space through:
- $p_{\theta}(z)$ the prior distribution of the latent space
- $p_{\theta}(x|z)$ the likelyhood
- $p_{\theta}(z|x)$ the posterior

Using the Bayes's theorem we could get:

$$p_{\theta}(z|x) = \frac{p_{\theta}(x|z)p_{\theta}(z)}{p_{\theta}(x)}$$

but the the computation is usually expensive if not intractable. However, it is possible to approximate the posterior:

$$ q_{\phi}(z|x)\simeq p_{\theta}(z|x)$$

### Variational Graph Autoencoders

Variational Graph Autoencoders [Kingma and Welling, 2016](https://arxiv.org/pdf/1611.07308.pdf) provide a framework extension to graph for VAEs.

Our problem could be formalized as follows: an undirected graph $\mathcal{G}=(\nu, \epsilon)$ with $N$ nodes and a features/attribute matrix $X\in\mathbb{R}^{N\times C}$. An adjacency matrix $A\in\mathbb{R}^{N\times N}$ with self-loops included. Assume that each node within the graph is associated to a latent variable $\in Z$ with $Z\in\mathbb{R}^{N\times F}$ and $F$ being the latent space dimension, we are interested in inferring the latent variables of nodes in the graph and decoding the edges.

Similarly to VAE, VGAE consist of an **encoder** $q_{\phi}(Z|A,X)$, a **decoder** $p_{\theta}(A|Z)$ and a prior $p(Z)$.
- The **encoder** tries to learn a distribution of latent variables associated with each node conditioning on the node features $X$ and $A$. One efficient option is to instantiate $q_{\phi}(Z|A,X)$ as a graph neural network where the learnable parameters are $\phi$. In particular, VGAE assumes a node-independent encoder so that the probabilities factorize: $$q_{\phi}(Z|A,X) = \prod_{i=1}^{N}q_{\phi}(z_{i}|A,X)$$ then, by neglecting from the third moment upwards of your distribution, the problem translates into: $$q_{\phi}(z_{i}|A,X)=\mathcal{N}(z_{i}|\mu_{i},diag(\sigma_{i}^2))$$ $$\mathbf{\mu},\mathbf{\sigma} = GCN_{\phi}(X,A)$$ Where $z_{i}, \mu_{i},\sigma_{i}$ are the i-th rows of the matrices $Z,\mu$ and $\sigma$. The mean and diagonal covariance are predicted by the encoder network, i.e. the $GCN$. For a two-layer $GCN$ we have: $$ H=\tilde{A}\sigma{(\tilde{A}XW_{1})}W_{2}$$ where $H\in\mathbb{R}^{N\times d_{H}}$ are the node representations (each node is associated with a size $d_{H}$ vector), $\tilde{A}=D^{-\frac{1}{2}}(A+I)D^{-\frac{1}{2}}$ is the normalized adjacency matrix as described by the [original 2016 GCN paper by Kipf and Welling](https://arxiv.org/abs/1609.02907). $\sigma$ is a pointwise nonlinearity (e.g. a ReLU) and $\{W_{1},W_{2}\}$ are trainable parameters containing the biases. Relying on the learned node representation, the distribution is computed as follows: $$q_{\phi}(Z|A,X) = \prod_{i=1}^{N}q_{\phi}(z_{i}|A,X)$$ $$q_{\phi}(z_{i}|A,X)=\mathcal{N}(z_{i}|\mu_{i},\sigma_{i}^2I)$$ $$\mu=MPL_{\mu}(H)$$ $$\log{\sigma}=MPL_{\sigma}{(H)}$$ Where $\mu_{i},\sigma_{i}$ are the i-th rows of the MPL predictions. Therefore, the set $\phi$ of parameters consist in the set of the trainable parameters of the twp MLPs and the aforementioned GCN. We remark that the NNs underlying each Gaussian ('GNN+MLP') are very powerful so that the conditional distributions are expressive in capturing the uncertainty of latent variables and computationally cheaper than other techniques.
- GVAEs often adopt a **prior** that remains fixed during the training. A common choice is a node-independent Gaussian as follows: $$p(Z)=\prod_{i}^{N}{p(z_{i})}$$ $$p(z_i)=\mathcal{N}(0,I)$$ Surely this prior can be substituted by more powerful models such as autoregressive models at the cost of more computational resources. Nevertheless, a simple prior like the one expressed before is usually the starting point to benchmark more complicated alternatives.
- The aim of a **decoder** is to construct a probability distribution over the graph and it's features/attributes conditioned on the latent variables, $p(\mathcal{G}|Z)$. One should always consider all the possible node permutations, each corresponding to an adjacency matrix with different rows orderings which leaves the graph unchanged: $$ p(\mathcal{G}|Z) = \sum_{P\in\prod_{\mathcal{G}}} {p(PAP^{T},PX|Z)}$$ but we'll neglect this discussion for the moment. A simple and popular construction of the probability distribution could be: $$ p(A,X|Z)=\prod_{i,j}p(A_{ij}|Z)\prod_{i=1}^{N}p(x_i|Z)$$ $$p(A_{ij}|Z)=Bernoulli(\Theta_{ij})$$ $$p(x_i|Z)=\mathcal{N}(\tilde{\mu}_{i},\tilde{\sigma}_i)$$ Where, once again, the parameters are learned through MLPs: $$\Theta_{ij}=MLP_{\Theta}([z_{i}||z_j])$$ $$\tilde{\mu}_{i}=MLP_{\tilde{\mu}}(z_i)$$ $$\tilde{\sigma}_{i}=MLP_{\tilde{\sigma}}(z_i)$$
- The **objective** of the GVAE is the evidence lower bound (ELBO): $$\max_{\theta,\phi}{\mathbb{E}_{q_{\phi}(Z|A,X)} {[\log{p_{\theta}(\mathcal{G}|Z)}} - KL(q_{\phi}(Z|A,X)||p(Z))]}$$ where the Kullback-Leibler divergence measures the divergence between two probability distributions

## Implementing a VGAE for link prediction

In [17]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

#Set libraries seed
np.random.seed(0)
torch.manual_seed(0)

#Set GPU device
device = torch.device('mps')

Firstly we implement a `transform` object that normalizes input feautures and directly performs tensor device conversion and randomly splits links in a 85/5/10 division.

In [29]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val = 0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
])
 #add_negative_train_samples = False -> model already performs negative sampling

Load Cora dataset with the transformation:

In [50]:
dataset = Planetoid('.', name = 'Cora', transform=transform)
train_data, val_data, test_data = dataset[0]

Let's import the models:

In [33]:
from torch_geometric.nn import GCNConv, VGAE

First of all we implement the encoder which should be composed of three GCN layers: 1 shared input layer, a second layer to approximate mean values $\mu_{i}$ and a third one for the variance (actually $\log{\sigma}$):

In [41]:
class Encoder(torch.nn.Module):

    def __init__(self, dim_in, dim_out):
        super().__init__()                 #overwrite the method of the parent class
        self.conv1 = GCNConv(dim_in, 2 * dim_out)
        self.conv_mu = GCNConv(2 * dim_out, dim_out)
        self.conv_logstd = GCNConv(2 * dim_out, dim_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

We initialize the VGAE layer with the Encoder as input. By default, the Decoder is set to be the inner product, which is actually what we need to perform link prediction.

In [42]:
model = VGAE(Encoder(dataset.num_features, 16)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

The `.train()` function firstly computes the embedding matrix Z through `model.encode()` which despite of the name, simply does sample embeddings from the learned distribution. Secondly, the ELBO loss is computed with `model.recon_loss()` (binary crossentropy loss) and model.kl_loss(). The decoder is implicitly called by the model to calculate the cross-entropy loss.

In [43]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index) + (1/train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

Then we could define a `test()` function which simply calls the VGAE's dedicated method:

In [46]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

In [48]:
for epoch in range(1000):
    loss = train()
    val_auc, val_ap = test(val_data)
    if epoch % 50 == 0:
        print(f'Epoch: {epoch} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}')

Epoch: 0 | Loss: 0.9460 | Val AUC: 0.8824 | Val AP: 0.8861
Epoch: 50 | Loss: 0.9071 | Val AUC: 0.8840 | Val AP: 0.8885
Epoch: 100 | Loss: 0.9188 | Val AUC: 0.8934 | Val AP: 0.8956
Epoch: 150 | Loss: 0.8944 | Val AUC: 0.8985 | Val AP: 0.8996
Epoch: 200 | Loss: 0.8903 | Val AUC: 0.9020 | Val AP: 0.9065
Epoch: 250 | Loss: 0.8860 | Val AUC: 0.9052 | Val AP: 0.9110
Epoch: 300 | Loss: 0.8736 | Val AUC: 0.8987 | Val AP: 0.9071
Epoch: 350 | Loss: 0.8747 | Val AUC: 0.8993 | Val AP: 0.9069
Epoch: 400 | Loss: 0.8747 | Val AUC: 0.8976 | Val AP: 0.9047
Epoch: 450 | Loss: 0.8688 | Val AUC: 0.8961 | Val AP: 0.9046
Epoch: 500 | Loss: 0.8606 | Val AUC: 0.8957 | Val AP: 0.9058
Epoch: 550 | Loss: 0.8534 | Val AUC: 0.8920 | Val AP: 0.9031
Epoch: 600 | Loss: 0.8703 | Val AUC: 0.8914 | Val AP: 0.9035
Epoch: 650 | Loss: 0.8496 | Val AUC: 0.8985 | Val AP: 0.9087
Epoch: 700 | Loss: 0.8500 | Val AUC: 0.8975 | Val AP: 0.9090
Epoch: 750 | Loss: 0.8537 | Val AUC: 0.8977 | Val AP: 0.9099
Epoch: 800 | Loss: 0.8514 |

In [51]:
test_auc, test_ap = test(test_data)
print(f'Test AUC: {test_auc:.4f} | Test AP: {test_ap:.4f}')

Test AUC: 0.9324 | Test AP: 0.9331


Finally we have out approximated adjacency matrix $\hat{A}$:

In [53]:
z = model.encode (test_data.x, test_data.edge_index)
Ahat = torch.sigmoid(z @ z.T)
Ahat

tensor([[0.8928, 0.4491, 0.5277,  ..., 0.5983, 0.8111, 0.7745],
        [0.4491, 0.9266, 0.8993,  ..., 0.5041, 0.6885, 0.6491],
        [0.5277, 0.8993, 0.9317,  ..., 0.4956, 0.8315, 0.7850],
        ...,
        [0.5983, 0.5041, 0.4956,  ..., 0.8586, 0.5083, 0.5493],
        [0.8111, 0.6885, 0.8315,  ..., 0.5083, 0.9305, 0.8997],
        [0.7745, 0.6491, 0.7850,  ..., 0.5493, 0.8997, 0.8702]],
       device='mps:0', grad_fn=<SigmoidBackward0>)

Training a VGAE is fast and outputs are easily understandable, however we know that GCNs are not the most expressive layers.